In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/gdrive')

In [ ]:
!ls -la /content/gdrive/My\ Drive/ColabData

In [ ]:
import pandas

articles = pandas.read_pickle('/content/gdrive/My Drive/ColabData/articles.pkl')

In [ ]:
print(articles.head(2))

In [ ]:
!pip install nltk
pip install --upgrade git+https://github.com/flairNLP/flair.git

In [ ]:
import torch
import flair
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
texts = articles.Text
tokenized_sentence_groups = [nltk.tokenize.sent_tokenize(text, 'german') for text in texts]

In [ ]:
article_sentences = []
for sentence_group in tokenized_sentence_groups:
  current_group = [flair.data.Sentence(sentence, language_code='ger') for sentence in sentence_group]
  article_sentences.append(current_group)


In [ ]:
del texts
print(len(article_sentences))

In [ ]:
flair.device

In [ ]:
tagger = flair.models.SequenceTagger.load('de-ner')

In [ ]:
current = 0
total = len(article_sentences)
predictions = []
for article in article_sentences:
  current_prediction = tagger.predict(article)
  lables = []
  entities = []
  for sentence_pred in current_prediction:
    ner_dict = sentence_pred.to_dict(tag_type='ner')
    lables += ner_dict['labels']
    entities += ner_dict['entities']
  predictions.append({'labels': lables, 'entities': entities})
  current += 1
  print(str(current) + "/" + str(total))
  if current % 1000 == 0:
    print("Saving file...")
    articles = pandas.DataFrame(articles['ID_Article'])
    articles = articles.assign(NEs=pandas.Series(predictions))
    articles.to_pickle('/content/gdrive/My Drive/ColabData/stored_ner_result_gpu_' + str(current) + '.pkl')


articles = pandas.DataFrame(articles['ID_Article'])
articles = articles.assign(NEs=pandas.Series(predictions))
articles.to_pickle('/content/gdrive/My Drive/ColabData/stored_ner_result_complete.pkl')
print("done")
